In [1]:
%cd ..

C:\Users\13661\PycharmProjects\KinshipVerification


In [2]:
import numpy as np
import torch
import torchvision
from torch.utils.data.dataloader import DataLoader

device = torch.device('cuda')

In [3]:
from dataset import Contrastive_pretrain

data_path = 'D:/文档/硕士/Thesis/UvA-NEMO_SMILE_DATABASE/aligned'

In [4]:
train_split = np.load('train.npy')
dataset = Contrastive_pretrain(data_path, train_split)
loader = DataLoader(
        dataset, batch_size=1, shuffle=False, num_workers=1)

In [5]:
from transformers import VideoMAEFeatureExtractor, VideoMAEModel, VideoMAEConfig
from datasets import load_dataset

feature_extractor = VideoMAEFeatureExtractor.from_pretrained("MCG-NJU/videomae-base", size=112)
config = VideoMAEConfig(image_size=112, num_frames=5)
model = VideoMAEModel.from_pretrained("C:\\Users\\13661\\PycharmProjects\\KinshipVerification\\ckpts", config=config).eval().to(device)


In [6]:
from tqdm import tqdm
import torch.nn.functional as F

outs = []
with torch.no_grad():
    with tqdm(loader, unit="batch") as data_loader:
        for batch in data_loader:
            batch_size = batch[0].size()[0]
            batch = torch.cat([batch[0], batch[1], batch[2]])
            batch = [list(b) for b in list(batch)]
            batch = feature_extractor(batch, return_tensors="pt")
            batch.to(device)

            sequence_output = model(**batch)[0]
            logits = sequence_output[:, 0]

            query = logits[:batch_size].squeeze(0).detach().cpu()
            positive_key = logits[batch_size:int(batch_size*2)].squeeze(0).detach().cpu()
            negative_key = logits[int(batch_size * 2):int(batch_size * 3)].squeeze(0).detach().cpu()
            
            outs.append([F.normalize(torch.abs(query - positive_key), dim=0).numpy(), 1])
            outs.append([F.normalize(torch.abs(query - negative_key), dim=0).numpy(), 0])

100%|███████████████████████████████████████████████████████████████████████████| 1154/1154 [00:38<00:00, 29.84batch/s]


In [7]:
import random
random.shuffle(outs)

X = [a[0] for a in outs]
Y = [a[1] for a in outs]

In [8]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True)

In [9]:
from sklearn import svm
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import recall_score

clf = svm.SVC()

# scoring = {'prec_macro': 'precision_macro','rec_macro': make_scorer(recall_score, average='macro')}
# scores = cross_validate(clf,X, Y,
#                        scoring=scoring, cv=kf,
#                        return_estimator=True)
clf.fit(X,Y)
clf.score(X,Y)
# print(scores)

0.9094454072790294

In [10]:
# print(scores['test_prec_macro'])

In [12]:
test_split = np.load('test.npy')
testset = Contrastive_pretrain(data_path, test_split)
testloader = DataLoader(
        testset, batch_size=1, shuffle=False, num_workers=1)

In [13]:
from tqdm import tqdm
import torch.nn.functional as F

t_outs = []
with torch.no_grad():
    with tqdm(testloader, unit="batch") as data_loader:
        for batch in data_loader:
            batch_size = batch[0].size()[0]
            batch = torch.cat([batch[0], batch[1], batch[2]])
            batch = [list(b) for b in list(batch)]
            batch = feature_extractor(batch, return_tensors="pt")
            batch.to(device)

            sequence_output = model(**batch)[0]
            logits = sequence_output[:, 0]

            query = logits[:batch_size].squeeze(0).detach().cpu()
            positive_key = logits[batch_size:int(batch_size*2)].squeeze(0).detach().cpu()
            negative_key = logits[int(batch_size * 2):int(batch_size * 3)].squeeze(0).detach().cpu()
                        
            t_outs.append([F.normalize(torch.abs(query - positive_key), dim=0).numpy(), 1])
            t_outs.append([F.normalize(torch.abs(query - negative_key), dim=0).numpy(), 0])

100%|█████████████████████████████████████████████████████████████████████████████| 516/516 [00:16<00:00, 31.95batch/s]


In [14]:
random.shuffle(t_outs)

test_x = [a[0] for a in t_outs]
test_y = [a[1] for a in t_outs]

In [15]:
clf.score(test_x, test_y)

0.45251937984496127